In [3]:
#추천시스템
# https://github.com/wikibook/ml-definitive-guide

# 확률적 경사 하강법을 이용한 행렬분해

In [4]:
import numpy as np

In [5]:
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

In [6]:
#분해 행렬 P&Q 초기화
num_users, num_items = R.shape

In [7]:
#col
num_users

4

In [8]:
#row
num_items

5

In [9]:
#잠재요인차원
k=3

In [10]:
#P와Q 행렬의 크기 지정, 정규 분포를 가진 임의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./k, size = (num_users,k))
Q = np.random.normal(scale=1./k, size = (num_items,k))

In [11]:
#행렬값의 위치 인덱스를 추출해 이 인덱스에 있는 실제 R 행렬값과 분해된 P,Q 를 이용해 다시 조합된 예측 행렬값의 RMSE 반환
from sklearn.metrics import mean_squared_error

In [12]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P,Q.T) #행렬 P와 Q의 내적으로 예측 R 생성
    
    #위치 인덱스를 추출 -> 실제 R행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [13]:
#SGD 기반 행렬 분해 
#steps = SGD 반복해서 업데이트할 횟수 
#learning_rate = SGD 학습률
#r_lambda = L2 regularization 계수
steps = 1000
learning_rate = 0.01
r_lambda = 0.01


#R>0인 행 위치, 열위치 값을 non_zeros 리스트에 저장
non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

In [14]:
#SGD 기법으로 P와Q 매트릭스 업데이트 ->50 회 반복 할때마다 오류값 출력
for step in range(steps):
    for i,j,r in non_zeros:
        eij = r - np.dot(P[i,:], Q[j,:].T) #실제 값과 예측값의 차이인 오류 값을 구함
        P[i,:] = P[i, :] + learning_rate*(eij*Q[j,:]-r_lambda*P[i,:]) #regularization을 반영한 SGD 업데이트 공식 적용
        Q[i,:] = Q[i, :] + learning_rate*(eij*Q[j,:]-r_lambda*Q[i,:])
        
        rmse = get_rmse(R,P,Q,non_zeros)
        if(step%50) == 0:
            print("### iterarion step:", step,"rmse:",rmse)

### iterarion step: 0 rmse: 3.264818196112103
### iterarion step: 0 rmse: 3.264370499465061
### iterarion step: 0 rmse: 3.261672121442261
### iterarion step: 0 rmse: 3.2624527494419877
### iterarion step: 0 rmse: 3.2627451924379254
### iterarion step: 0 rmse: 3.2624024947875943
### iterarion step: 0 rmse: 3.2607390798068367
### iterarion step: 0 rmse: 3.2595221381444444
### iterarion step: 0 rmse: 3.260954730435898
### iterarion step: 0 rmse: 3.2609508165214223
### iterarion step: 0 rmse: 3.2607914544151466
### iterarion step: 0 rmse: 3.260595390737092
### iterarion step: 50 rmse: 1.6672341428098716
### iterarion step: 50 rmse: 1.6745479049778667
### iterarion step: 50 rmse: 1.6572330212582147
### iterarion step: 50 rmse: 1.6520681864262174
### iterarion step: 50 rmse: 1.6534861686695055
### iterarion step: 50 rmse: 1.6534469058766692
### iterarion step: 50 rmse: 1.6525286596767195
### iterarion step: 50 rmse: 1.652526781210148
### iterarion step: 50 rmse: 1.6559357535642625
### iterar

### iterarion step: 700 rmse: 1.4147716877761511
### iterarion step: 700 rmse: 1.4121022885272216
### iterarion step: 700 rmse: 1.4158526586248263
### iterarion step: 700 rmse: 1.414855817585214
### iterarion step: 700 rmse: 1.416508996859151
### iterarion step: 700 rmse: 1.412397236433255
### iterarion step: 700 rmse: 1.4146779195534003
### iterarion step: 700 rmse: 1.415487799863068
### iterarion step: 750 rmse: 1.4363085424148079
### iterarion step: 750 rmse: 1.4368491445080773
### iterarion step: 750 rmse: 1.4368793824635695
### iterarion step: 750 rmse: 1.4361007744923164
### iterarion step: 750 rmse: 1.4365116463946859
### iterarion step: 750 rmse: 1.4339120005085042
### iterarion step: 750 rmse: 1.4378128729862163
### iterarion step: 750 rmse: 1.4366152326552926
### iterarion step: 750 rmse: 1.4379370248184482
### iterarion step: 750 rmse: 1.433619871439622
### iterarion step: 750 rmse: 1.4363110354004855
### iterarion step: 750 rmse: 1.4373246573722052
### iterarion step: 800 r

In [15]:
#P*Q.T 예측 행렬 출력
pred_matrix = np.dot(P,Q.T)
print('예측 행렬:\n', np.round(pred_matrix,3))

예측 행렬:
 [[ 3.784  0.769  2.095  2.309 -0.55 ]
 [ 2.389  5.173  3.897  2.185 -0.48 ]
 [ 2.449  4.595  3.704  1.922 -0.357]
 [ 3.933  1.239  2.366  2.581 -0.647]]


# 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

In [16]:
#장르 속성을 이용한 영화 콘텐츠기반 필터링

import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies =pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [17]:
#사용할 컬럼만 추출
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

In [18]:
#해당 컬럼 형태 확인
pd.set_option('max_colwidth', 100) #출력될 표의 크기 최대 100 설정
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [19]:
#genres와 keywords 'key'값(='name') 추출 -> 파이썬 리스트 객체로 추출
from ast import literal_eval # literal_eval(): 문자열을 list 객체(list[dict1,dict2])로 변환
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [20]:
#key값('name')만 리스트 객체로 추출
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [21]:
#장르별 유사도 측정 방법 => 장르를 문자별로 변경한뒤 countVectorizer로 피처벡터화한 행렬 데이터 값을 코사인 유사도로 비교
#문자열로 변화된 genres 칼럼을 count  기반으로 피처 벡터화
#genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도와 비교 <- 데이터 세트의 레코드 별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체 생성
#장르 유사도가 높은 영화중 평점이 높은 순으로 영화 추천

#피처 벡터 행렬 생성 <- 리스트 객체 내 개별 값을 연속된 문자열로 변환:('구분문자').join(리스트객체)
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백 문자로 word 단위가 구분되는 문자열로 변환 -> genres_listeral 칼럼으로 저장
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x :('').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 1394)


In [22]:
#코사인유사도계산
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1. 0. 0. ... 0. 0. 0.]]


In [23]:
#비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값 추출
#0번 레코드의 비교 행 위치 인덱스 값 추출
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1] #높은 순으로 정렬된 비교행 위치 인덱스 값
print(genre_sim_sorted_ind[:1]) #0번째 레코드의 비교행 위치 인덱스 값 추출

[[   0  870   46 ... 3172 3173 2401]]


In [30]:
#find_sim_movie() : 장르 유사도에 따라 영화 추천 함수 생성
#input: movies_df,genre_sim_sorted_ind(레코드별 장르 코사인 유사도 인덱스),고객이 선정한 추천 기준이 되는 영화 제목, 추천하 영화 건수
#output : 추천 영화 정보를 가지는 DataFrame 반환

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name] #'title'칼럼이 입력된 title_name 값인 DataFrame 추출
    title_index = title_movie.index.values #title_named을 가진 index 객체를 ndarray로 반환
    similar_indexs = sorted_ind[title_index, :(top_n)] 
    #sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 인덱스 추출
    
    print(similar_indexs)
    similar_indexs = similar_indexs.reshape(-1)
    
    return df.iloc[similar_indexs]

In [31]:
#'The Godfather'와 장르별로 유사한 영화 10개 추천
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[3636  883 1149 3337 1464 1243 1847 3866 2582 4065]]


,title,vote_average
3636,Light Sleeper,5.7
883,Catch Me If You Can,7.7
1149,American Hustle,6.8
3337,The Godfather,8.4
1464,Black Water Transit,0.0
1243,Mean Streets,7.2
1847,GoodFellas,8.2
3866,City of God,8.1
2582,The Place Beyond the Pines,6.8
4065,Mi America,0.0


In [33]:
#더 많은 후보군을 생성 후 영화 평점(=vote_average)에 따라 필터링하여 최종 추천
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [34]:
#가중평점을 이용하여 평가 횟수에대한 가중치가 부여된 평점 생성
#가중평점 = (v/(v+m)*R+(m/(v+m))*C)
#v = 개별 영화를 투표한 횟수
#R = 개별 영화의 평균 평점
#C = 전체 영화에 대한 평균 평점
C=movies_df['vote_average'].mean()
#m =  최소 투표 횟수
m=movies_df['vote_count'].quantile(0.6)

print('C:',round(C,3),'m:',round(m,3))

C: 6.092 m: 370.2


In [ ]:
#weightes_vote_average() 생성 : 레코드별 가중평점 반환

percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C=movies_df['vote_average'].mean()

def 